## Report generation

In [2]:
import os, string, sys
import numpy as np
import pyaml, json
from astropy.io import ascii
from IPython.display import HTML, display

import logging
logger = logging.getLogger()

import rfinder as rfinder

print '\t+------+\n\t  Done \n\t+------+'

	+------+
	  Done 
	+------+


In [3]:
print '\t+------+\n\t RFInder \n\t Parameter File \n\t+------+'


import rfi
rfi = rfi.rfi()

rfi_par = rfinder.rfinder()



print '\t+------+\n\t Parameters Loaded \n\t+------+'

INFO:rfinder:	 ... Reading parameter file ... 



	+------+
	 RFInder 
	 Parameter File 
	+------+
	+------+
	 Parameters Loaded 
	+------+


In [4]:
# Set working directory & filenames
rfi_par.cfg_par['general']['workdir'] = '/Users/maccagni/Projects/RFI/rfinder_test/'

# Enable / disable modules
rfi_par.cfg_par['rfi']['enable'] = True
rfi_par.cfg_par['rfi']['RFInder_mode'] = 'use_flags'

rfi_par.cfg_par['plots']['enable'] = True

#MSFILE
rfi_par.cfg_par['general']['msname'] = ['fa1_conc.ms']
rfi_par.cfg_par['general']['field'] = 1


# CHUNKS
#create also tables of data binned in frequency and time
rfi_par.cfg_par['rfi']['chunks']['time_enable'] = False
rfi_par.cfg_par['rfi']['chunks']['time_step'] = 10 #in minutes

rfi_par.cfg_par['rfi']['chunks']['spw_enable'] = True
rfi_par.cfg_par['rfi']['chunks']['spw_width'] = 20 #MHz


#Options
#set separation in long and short baselines
rfi_par.cfg_par['rfi']['baselines_cut'] = 200 #m

#plotting options
rfi_par.cfg_par['plots']['long_short'] = False
rfi_par.cfg_par['plots']['plot_noise'] = 'rfi'

rfi_par.set_dirs()

print '\t+------+\n\t Parameters modified \n\t+------+'

	+------+
	 Parameters modified 
	+------+


In [5]:
from jinja2 import FileSystemLoader, Environment
rfi.load_from_ms(rfi_par.cfg_par,0)
rfi.baselines_from_ms(rfi_par.cfg_par)

print '\t+------+\n\t Html stuff loaded \n\t+------+'

INFO:rfi:	 ... Field, Antenna & Bandwidth Info ...

INFO:rfi:	Field with name Fornax (Field ID = 1)
INFO:rfi:	Total number of antennas:	16
INFO:rfi:	Antenna names:		[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
INFO:rfi:	Channel Width [kHz]:	208.984375
INFO:rfi:	Start         [GHz]:	1.29716601563
INFO:rfi:	End           [GHz]:	1.44345507812

INFO:rfinder_stats:	 ...  Observing time Info ... 

INFO:rfinder_stats:	 Start date: 17Sep07:22:28:15
INFO:rfinder_stats:	 End date  : 17Sep08:07:58:31 




Successful readonly open of default-locked table /Users/maccagni/Projects/RFI/rfinder_test/fa1_conc.ms/FIELD: 9 columns, 4 rows
Successful readonly open of default-locked table /Users/maccagni/Projects/RFI/rfinder_test/fa1_conc.ms/ANTENNA: 8 columns, 16 rows
Successful readonly open of default-locked table /Users/maccagni/Projects/RFI/rfinder_test/fa1_conc.ms/SPECTRAL_WINDOW: 14 columns, 1 rows
Successful readonly open of default-locked table /Users/maccagni/Projects/RFI/rfinder_test/fa1_conc.ms: 27 columns, 475920 rows
Successful readonly open of default-locked table /Users/maccagni/Projects/RFI/rfinder_test/fa1_conc.ms: 27 columns, 475920 rows


INFO:rfinder_stats:	 ... Altitude/Azimuth info ... 

INFO:rfinder_stats:		 ... Altitude = [ 37.57683783] deg
INFO:rfinder_stats:		 ... Azimuth = [ 115.71918503] deg

INFO:rfinder_stats:	 ... Alt/Az done ... 

INFO:rfinder_stats:	 ...  Predicting natural r.m.s. ... 

INFO:rfinder_stats:	 Assumptions on meerkat telescope
INFO:rfinder_stats:		Dish diameter = 13.5 m
INFO:rfinder_stats:		 ... and SEFD = 578.710952 Jy
INFO:rfinder_stats:		 ... and Tsys = 30.0 K
INFO:rfinder_stats:	 Properties of observation
INFO:rfinder_stats:		 Total number of baselines = 120.0
INFO:rfinder_stats:		 Total number of channels = 701
INFO:rfinder_stats:		 Observing time on source = 6.65274 h (2 polarisations)

INFO:rfinder_stats:	 Stokes I natural r.m.s.       = 3.734e-01 mJy/b 
INFO:rfinder_stats:	 ... Natural r.m.s. predicted ... 

INFO:rfi:	 ... info from MS file loaded  


INFO:rfi:	 ... Sorting baselines ... 

INFO:rfinder_stats:	 ... Baseline info ... 

INFO:rfinder_stats:		 Maximum baseline length     m 

	+------+
	 Html stuff loaded 
	+------+


In [27]:
# Content to be published
import numpy as np
# Configure Jinja and ready the template
env = Environment(
    loader=FileSystemLoader('/Users/maccagni/notebooks/rfinder/report_templates/')
)
template = env.get_template('rfinder_template.html')

rfi_par.cfg_par['general']['fieldname'] = 'Fornax A'
#base_template = env.get_template('report.html')
# Content to be published
title = 'RFI report: {0:s}'.format(rfi_par.cfg_par['general']['msname'])

imagename1 = '/Users/maccagni/Projects/RFI/rfinder_test/rfi/plots/altaz/AltAZ_rfi1297-1317MHz.png'
data_uri1 = open(imagename1, 'rb').read().encode('base64').replace('\n', '')

imagename2 = '/Users/maccagni/Projects/RFI/rfinder_test/rfi/plots/rfi_base_full.png'
data_uri2 = open(imagename2, 'rb').read().encode('base64').replace('\n', '')

with open(rfi_par.cfg_par['general']['rfidir']+'report.html', "w") as f:
    lenghts = np.array([rfi_par.cfg_par['rfi']['baseline_lenghts']])+0.
    f.write(template.render(
        title=title,
        fieldname=rfi_par.cfg_par['general']['fieldname'],
        field=rfi_par.cfg_par['general']['field'],
        totchans = int(rfi_par.cfg_par['rfi']['total_channels']),
        chan_widths=round(rfi_par.cfg_par['rfi']['chan_widths']/1e3,4),
        lowfreq=round(rfi_par.cfg_par['rfi']['lowfreq']/1e6,3),
        highfreq=round(rfi_par.cfg_par['rfi']['highfreq']/1e6,3),
        startdate = ('{0:%y}{0:%b}{0:%d} {0:%X}'.format(rfi_par.cfg_par['rfi']['startdate'].datetime)),
        enddate =   ('{0:%y}{0:%b}{0:%d} {0:%X}'.format(rfi_par.cfg_par['rfi']['enddate'].datetime)),
        nant = rfi_par.cfg_par['rfi']['nant'],
        ant_names = rfi_par.cfg_par['rfi']['ant_names'],
        maxbase = str(np.round(lenghts[0][-1],0)),
        minbase = str(np.round(lenghts[0][0],0)),
        totbase = rfi_par.cfg_par['rfi']['number_baseline'],
        exptime = np.round(rfi_par.cfg_par['rfi']['exptime'],2),
        polnum = rfi_par.cfg_par['rfi']['polnum'],
        noise = np.round(rfi_par.cfg_par['rfi']['theo_rms'][0]*1e3,5),
        img_tag1 = '<img class="a" src="data:image/png;base64,{0}">'.format(data_uri1),
        img_tag2 = '<img class="b" src="data:image/png;base64,{0}">'.format(data_uri2)

    ))

print '\t+------+\n\t Html stuff loaded \n\t+------+'

	+------+
	 Html stuff loaded 
	+------+
